# Summary Prediction Evaluation

In [1]:
!pip install -q transformers
!pip install -q --upgrade datasets
!pip install -q rouge_score
!pip install -q sentencepiece
!pip install -q torchviz
!pip install -q cloud-tpu-client
!pip install -q torch-xla
!pip install -q transformers[torch]
!pip install -q evaluate
!pip install -q bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.6.3 requires google-api-python-client>=1.12.5, but you have google-api-python-client 1.8.0 which is incompatible.
earthengine-api 0.1.381 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatib

In [2]:
import os
import time
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from pprint import pprint

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import pipeline
from datasets import load_metric
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets
import evaluate
from evaluate import evaluator
from evaluate import SummarizationEvaluator
import nltk
nltk.download('punkt')
import torch
from torch.utils.data import Dataset, DataLoader
from torchviz import make_dot

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.utils import logging

seed = random.seed(42)
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
drive.mount('/content/drive', force_remount=True)
#! cd drive
os.chdir("/content/drive/MyDrive/Data 266 Project")
#!ls

Mounted at /content/drive


# Dataset Load

In [ ]:
# Maximum length of content and summary covers over 99% of the posts
max_len_content = 1024
max_len_summary = 130

In [ ]:
rouge = load_metric("rouge")

<ipython-input-5-6f1857dc142f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [ ]:
# dataset_train = load_dataset('csv', data_files='df_train.csv', split='train')
dataset_test = load_dataset('csv', data_files='df_test.csv', split='train')
# dataset_val = load_dataset('csv', data_files='df_valid.csv', split='train')
# dataset_dict = DatasetDict({"train": dataset_train, "test": dataset_test, 'validation': dataset_val})
# dataset_dict
dataset_small_pred = DatasetDict.load_from_disk('dataset_small_pred')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset_test[0]

{'subreddit': 'AskReddit',
 'id': 'c5hefrx',
 'content': 'I\'ve told this one before, but it was never noticed, so: \n I snuck into a sold out concert with my friends once. \n My friend, who we\'ll call E, texted me earlier that day asking if I wanted to go see a concert that night on a whim. Usually we planned these things far out in advance, but seeing as tickets were still on sale, we figured "Why not?" and drove the hour ride up to the town, paid for $5 parking across the street. Got in line with some friends of ours that we recognized, and figured we\'d buy our ticket once we got up to the front. \n However, the venue failed to specify how many tickets were left, or that they were even running low. With no warning, one of the security guards shouted that that nights show was "officially sold out, if you don\'t have a ticket - go home." \n E and I were understandably pissed, though looking back, we should\'ve just marched up to the front and bought our tickets in the first place. W

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
})

In [ ]:
def preprocess_data(examples, tokenizer):
  inputs = ["summarize: " + text for text in examples['content']]
  model_inputs = tokenizer(inputs, max_length = max_len_content, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], max_length = max_len_summary, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

def tokenize_dataset(input_tokenizer):
  return dataset_test.map(preprocess_data, batched=True, fn_kwargs={'tokenizer': input_tokenizer})

In [ ]:
def generate_summaries_with_pipeline(examples, input_pipeline, name):
  res = input_pipeline(examples['content'], max_length=max_len_summary, truncation=True)
  return {'generated_' + name: [obj['summary_text'] for obj in res]}

In [ ]:
list(set(dataset_test['subreddit']))

['funny', 'AskReddit', 'worldnews']

# Generate a smaller subset

In [ ]:
dataset_gen = dataset_test
if (os.path.isfile('./df_predictions.csv')):
  dataset_gen = load_dataset('csv', data_files='df_predictions.csv', split='train')

In [ ]:
dataset_funny_gen = dataset_gen.filter(lambda example: example['subreddit'] == 'funny').shuffle(seed=42).select(range(500))
dataset_worldnews_gen = dataset_gen.filter(lambda example: example['subreddit'] == 'worldnews').shuffle(seed=42).select(range(500))
dataset_askreddit_gen = dataset_gen.filter(lambda example: example['subreddit'] == 'AskReddit').shuffle(seed=42).select(range(500))

In [ ]:
dataset_small_gen = DatasetDict({
    'funny': dataset_funny_gen,
    'worldnews': dataset_worldnews_gen,
    'askreddit': dataset_askreddit_gen
})

In [ ]:
dataset_small_gen

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
})

In [ ]:
dataset_small_gen.save_to_disk('dataset_small_pred')

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

# T5 Base Eval

In [ ]:
summarizer = pipeline("summarization", model='t5-small')
dataset_gen = dataset_gen.map(generate_summaries_with_pipeline, batched=True, fn_kwargs={'input_pipeline': summarizer, 'name': 't5_base'})

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
dataset_gen.to_csv("df_predictions.csv")

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

5003049

In [ ]:
dataset_gen['generated_t5_base'][0]

'we snuck into a sold out concert with my friends that night . the venue failed to specify how many tickets were left, or that they were even running low . a security guard shouted that the show was "officially sold out, if you don\'t have a ticket"'

# BART Base Eval

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base'],
        num_rows: 500
    })
})

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-base")
dataset_small_pred = dataset_small_pred.map(generate_summaries_with_pipeline, batched=True, batch_size=100, fn_kwargs={'input_pipeline': summarizer, 'name': 'bart_base'})

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset_small_pred.save_to_disk('dataset_small_pred1')

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

# PEGASUS Base Eval

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base'],
        num_rows: 500
    })
})

In [ ]:
summarizer = pipeline("summarization", model="google/pegasus-xsum")
dataset_small_pred = dataset_small_pred.map(generate_summaries_with_pipeline, batched=True, batch_size=100, fn_kwargs={'input_pipeline': summarizer, 'name': 'pegasus_base'})

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset_small_pred.save_to_disk('dataset_small_pred2')

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

# T5 Fine tuned Eval

In [ ]:
dataset_small_pred['funny']['content'][0]

'In the past couple of decades, writers have started to portray Batman as the opposite of his villains, namely the Joker. He\'s just as determined, he\'s just as much a believer in "ends justifying means" and he\'s just as ruthless. The only thing that separates him from his Rogues Gallery is his rule about killing, at least directly. I\'d be terrified of a Batman who did not have that rule. \n I\'m going to focus on Batman v Joker because that\'s the central one. The fundamental thing about this conflict is that it\'s not superhero versus supervillain: it\'s the clashing of ideologies. Both people came to a point where they faced loss and this loss drove both men down different paths. Batman believes that people are fundamentally good and that they just need a reminder, even if that reminder is a guy who stalks the night and terrifies the piss out of you. The Joker believes that people are fundamentally evil and (especially in the movies) the only thing that keeps us from killing each

In [ ]:
summarizer = pipeline("summarization", model='./models/t5-fine-tune')
dataset_small_pred = dataset_small_pred.map(generate_summaries_with_pipeline, batched=True, batch_size=100, fn_kwargs={'input_pipeline': summarizer, 'name': 't5_finetuned'})

NameError: ignored

In [ ]:
dataset_small_pred.save_to_disk('dataset_small_pred3')

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

# BART Fine tuned Eval

In [ ]:
summarizer = pipeline("summarization", model='./models/bart-fine-tune')
dataset_small_pred = dataset_small_pred.map(generate_summaries_with_pipeline, batched=True, batch_size=100, fn_kwargs={'input_pipeline': summarizer, 'name': 'bart_finetuned'})

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset_small_pred.save_to_disk('dataset_small_pred1')

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
})

# ROUGE Scores

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned'],
        num_rows: 500
    })
})

In [ ]:
dataset_small_pred['funny']['generated_t5_base'][0]

"writers have started to portray Batman as the opposite of his villains, namely the Joker . john avlon: it's the clashing of ideologies, not superhero versus supervillain . they say if he does something so heinous, so evil, so wrong, he'll kill him ."

In [ ]:
dataset_small_pred['funny']['summary'][0]

"This inability or unwillingness to kill is Batman's kryptonite, if you want to put it in superhero terms. He  knows  that if he kills the Joker, murders Bane, beheads Two-Face, and executes Ra's, the world would be much better off. But he'll be destroyed."

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
def compute_rouge(pred_to_compute):
  splits = ['funny', 'askreddit', 'worldnews']
  df_splits = []
  for split in splits:
    predictions = dataset_small_pred[split][pred_to_compute]
    references = dataset_small_pred[split]['summary']
    results = rouge.compute(predictions=predictions,
                            references=references)
    results['split'] = split
    df_splits.append(pd.DataFrame.from_dict({key: [val] for key, val in results.items()}))
  combined = concatenate_datasets([dataset_small_pred['funny'], dataset_small_pred['askreddit'], dataset_small_pred['worldnews']])
  predictions = combined[pred_to_compute]
  references = combined['summary']
  results = rouge.compute(predictions=predictions,
                          references=references)
  results['split'] = 'all'
  df_splits.append(pd.DataFrame.from_dict({key: [val] for key, val in results.items()}))
  return pd.concat(df_splits)

In [ ]:
df_scores_t5 = compute_rouge('generated_t5_base')
df_scores_t5

,rouge1,rouge2,rougeL,rougeLsum,split
0,0.173143,0.025486,0.119443,0.122003,funny
0,0.190564,0.037701,0.135745,0.138075,askreddit
0,0.188431,0.033319,0.128181,0.131884,worldnews
0,0.184147,0.032032,0.127745,0.130701,all


In [ ]:
df_scores_t5['model'] = 't5_base'

In [ ]:
df_scores_t5

,rouge1,rouge2,rougeL,rougeLsum,split,model
0,0.173143,0.025486,0.119443,0.122003,funny,t5_base
0,0.190564,0.037701,0.135745,0.138075,askreddit,t5_base
0,0.188431,0.033319,0.128181,0.131884,worldnews,t5_base
0,0.184147,0.032032,0.127745,0.130701,all,t5_base


In [ ]:
model_names = ['t5_base', 'bart_base', 'pegasus_base', 't5_finetuned', 'bart_finetuned']
df_model_scores = []
for model_name in model_names:
  df_scores = compute_rouge(f'generated_{model_name}')
  df_scores['model'] = model_name
  df_model_scores.append(df_scores)
df_all_model_scores = pd.concat(df_model_scores)
df_all_model_scores

,rouge1,rouge2,rougeL,rougeLsum,split,model
0,0.173338,0.025451,0.119432,0.121784,funny,t5_base
0,0.190568,0.037586,0.135747,0.137960,askreddit,t5_base
0,0.188436,0.033424,0.128410,0.132045,worldnews,t5_base
0,0.184232,0.032112,0.127878,0.130605,all,t5_base
0,0.162902,0.029101,0.106242,0.109320,funny,bart_base
0,0.156228,0.033013,0.105969,0.108544,askreddit,bart_base
0,0.181360,0.032678,0.114179,0.119450,worldnews,bart_base
0,0.166833,0.031548,0.108824,0.112547,all,bart_base
0,0.129936,0.018162,0.104170,0.104948,funny,pegasus_base
0,0.134402,0.021918,0.104362,0.105557,askreddit,pegasus_base


In [ ]:
df_all_model_scores.to_csv('model_scores.csv')

# BERTScore

In [ ]:
bertscore = evaluate.load('bertscore')

In [ ]:
def compute_bertscore(pred_to_compute):
  splits = ['funny', 'askreddit', 'worldnews']
  df_splits = []
  for split in splits:
    predictions = dataset_small_pred[split][pred_to_compute]
    references = dataset_small_pred[split]['summary']
    results = bertscore.compute(predictions=predictions,
                            references=references,
                                lang='en')
    scores = pd.DataFrame.from_dict({key: val for key, val in results.items() if key != 'hashcode'})
    agg = scores.mean().to_frame().T
    agg['split'] = split
    df_splits.append(agg)
  combined = pd.concat(df_splits).drop(['split'], axis=1)
  combined = combined.mean().to_frame().T
  combined['split'] = 'all'
  df_splits.append(combined)
  return pd.concat(df_splits)

In [ ]:
dataset_small_pred

DatasetDict({
    funny: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    worldnews: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
    askreddit: Dataset({
        features: ['subreddit', 'id', 'content', 'summary', 'generated_t5_base', 'generated_bart_base', 'generated_pegasus_base', 'generated_t5_finetuned', 'generated_bart_finetuned'],
        num_rows: 500
    })
})

In [ ]:
model_names = ['t5_base', 'bart_base', 'pegasus_base', 't5_finetuned', 'bart_finetuned']
df_model_bertscores = []
for model_name in model_names:
  df_scores = compute_bertscore(f'generated_{model_name}')
  df_scores['model'] = model_name
  df_model_bertscores.append(df_scores)
  pd.concat(df_model_bertscores).to_csv('model_bertscores.csv')
df_all_model_bertscores = pd.concat(df_model_bertscores)
df_all_model_bertscores

,precision,recall,f1,split,model
0,0.830709,0.849197,0.839734,funny,t5_base
0,0.833564,0.855590,0.844330,askreddit,t5_base
0,0.833409,0.849119,0.841068,worldnews,t5_base
0,0.832561,0.851302,0.841711,all,t5_base
0,0.818015,0.852920,0.834968,funny,bart_base
0,0.818230,0.855977,0.836569,askreddit,bart_base
0,0.820082,0.853413,0.836273,worldnews,bart_base
0,0.818776,0.854103,0.835937,all,bart_base
0,0.851551,0.840494,0.845713,funny,pegasus_base
0,0.852103,0.843310,0.847430,askreddit,pegasus_base


In [ ]:
df_all_model_bertscores.to_csv('model_bertscores.csv')

In [ ]:
pprint(dataset_small_pred['askreddit'][111])

{'content': "Since I was about nine years old, I've been having a dream where "
            "I'm walking through the upstairs auditorium of my elementary "
            'school which is filled with kids I used to go to school with, '
            'most of whom became gang bangers or drug addicts. Eventually, I '
            'get to the back and I sit on a ledge in this big empty storage '
            "room, where there's only one girl. \n"
            ' The girl is beautiful, and I always flirt with her and talk her '
            'up in my dreams and she looks familiar to me. Before I can kiss '
            "her I always wake up, but I know she's into me and I have this "
            'dream about once a month. \n'
            ' Anyway, I met this girl drunk at a party who looked really '
            "familiar to me, which I told her. She said she'd never met me "
            'before but we talked a little and shared a cigarette before she '
            'started puking in the bathroom. I 

In [ ]:
dataset_small_pred['askreddit'][111]

{'subreddit': 'AskReddit',
 'id': 'c3gpuny',
 'content': "Since I was about nine years old, I've been having a dream where I'm walking through the upstairs auditorium of my elementary school which is filled with kids I used to go to school with, most of whom became gang bangers or drug addicts. Eventually, I get to the back and I sit on a ledge in this big empty storage room, where there's only one girl. \n The girl is beautiful, and I always flirt with her and talk her up in my dreams and she looks familiar to me. Before I can kiss her I always wake up, but I know she's into me and I have this dream about once a month. \n Anyway, I met this girl drunk at a party who looked really familiar to me, which I told her. She said she'd never met me before but we talked a little and shared a cigarette before she started puking in the bathroom. I got alot more drunk and at the end of the night I asked her if she'd like to make out, but she denied me. \n She looked me up on Facebook a few days l